In [2]:
import pandas as pd
import numpy as np
import scipy as sp
import os
import json
import pprint
%matplotlib inline
import tweepy
import networkx as nx
import random
from pyvis.network import Network
print("imported completed! ciaoooo")

imported completed! ciaoooo


In [4]:
consumer_key = "GvDHFIS3fpCfZR2hwEGhE5oCZ"
consumer_secret = "LCwgUZgFOjJppd4jg6Cf4mgohUTc6Ro7Yaals87LdNLmS3d9zB"
access_token = "810960428385890304-aVlOFR9EExKdDPZCsh5EDN5Nm9SUmb9"
access_secret = "XJkr9dKABPkSvRIgpyQeNapg2o6ldP4VoNbTids9QXGWT"
bearer_token = "AAAAAAAAAAAAAAAAAAAAAA8UJQEAAAAAOD26yUUUPHF9zeAe56TtzGYUK8Q%3DRh6u9TAI7T7b5TJABTBOP5WwWSL0Cbvv5a6JJwg8MursyOOt9L"

In [5]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)
api = tweepy.API(auth, wait_on_rate_limit = True, wait_on_rate_limit_notify = True)
if api.verify_credentials :
  print ('Authentication completed successfully!')

Authentication completed successfully!


In [6]:
data_folder =  "data"
def serialize_json    (folder, filename, data):
  if not os.path.exists(folder):
    os.makedirs(folder, exist_ok=True)
  with open(f"{folder}/{filename}", "w", encoding="utf-8") as f:
    json.dump(data, f, ensure_ascii=False, indent=4)
  print(f"Data serialized to path: {folder}/{filename}")

def read_json(path):
  if os.path.exists(path):
    with open(path, "r", encoding="utf8") as file:
      data = json.load(file)
    print(f"Data read fron path: {path}")
    return data
  else:
    print(f"No data found at path: {path}")
    return {}

def usersID (users):
    usersID = []
    for user in users:
        json_data = api.get_user(user)._json
        usersID.append(json_data["id"])
    return usersID

def download_followers(idUser, quantity):
  followers_of_user = []
  for follower in tweepy.Cursor(
        api.followers,
        id = user,
        skip_status=True,
        include_user_entities = False
    ).items(quantity):
        #json_data = follower._json
        followers_of_user.append(follower._json["id"])
  return followers_of_user

def download_following(user, quantity):
  friends_of_user = []
  for friend in tweepy.Cursor(
        api.friends,
        id = user,
        skip_status=True,
        include_user_entities = False
    ).items(quantity):
        json_data = friend._json
        friends_of_user.append(friend._json["id"])
  return friends_of_user

In [ ]:
users = ["lucabazzetto","Tomve99"]
#usersID = usersID(users)
quantity = 800
followers = {}
for user in users:
    followers_of_user = []
    for follow in tweepy.Cursor(
        api.followers,
        screen_name = user,
        skip_status=True,
        include_user_entities = False
    ).items(quantity):
        json_data = follow._json
        if json_data not in followers_of_user:
            followers_of_user.append(json_data["screen_name"])
    followers[user] = followers_of_user
serialize_json(data_folder,"followersLucaThomas.json", followers)

Data serialized to path: data/followersLucaThomas.json


In [ ]:
users = ["lucabazzetto","Tomve99"]
quantity = 800
friends = {}
for user in users:
    friends_of_user = []
    for friend in tweepy.Cursor(
        api.friends,
        screen_name = user,
        skip_status=True,
        include_user_entities = False
    ).items(quantity):
        json_data = friend._json
        friends_of_user.append(friend._json["screen_name"])
    friends[user] = friends_of_user
serialize_json(data_folder,"friendsLucaThomas.json", friends)

Data serialized to path: data/friendsLucaThomas.json


In [ ]:

def random_json_with_control(json_array, quantity, all):
    a = quantity
    array = []
    while(a!=0):
        item = random.choice(json_array)
        if item not in array and item not in all:
            array.append(item)
            a = a-1
    return array

followersALL = read_json(f"{data_folder}/followersLucaThomas.json")
fiveFollowers = []

# Estraggo per ogni utente 5 followers
for user in followersALL:
    fiveFollowers.extend(random_json_with_control(followersALL[user], 3, fiveFollowers))

quantity = 10
followersOfFollowers = {}

for user in fiveFollowers:
    followers_of_user = []
    for follower in tweepy.Cursor(
        api.followers,
        screen_name = user,
        skip_status=True,
        include_user_entities = False
    ).items(quantity):
        json_data = follower._json
        followers_of_user.append(follower._json["screen_name"])
    followersOfFollowers[user] = followers_of_user

serialize_json(data_folder,"followersOfFollowers.json", followersOfFollowers)

Data read fron path: data/followersLucaThomas.json
Data serialized to path: data/followersOfFollowers.json


In [ ]:
friendsALL = read_json(f"{data_folder}/friendsLucaThomas.json")
fiveFriends = []

# Estraggo per ogni utente 5 followers
for user in friendsALL:
    fiveFriends.extend(random_json_with_control(friendsALL[user], 5, fiveFriends))

quantity = 10
friendsOfFriends = {}
for user in fiveFriends:
    friends_of_user = []
    for friend in tweepy.Cursor(
        api.friends,
        screen_name = user,
        skip_status=True,
        include_user_entities = False
    ).items(quantity):
        json_data = friend._json
        friends_of_user.append(friend._json["screen_name"])
    friendsOfFriends[user] = friends_of_user
serialize_json(data_folder,"friendsOfFriends.json", friendsOfFriends)

Data read fron path: data/friendsLucaThomas.json
Data serialized to path: data/friendsOfFriends.json


In [7]:
allFriends = {}
allFriends.update(read_json(f"{data_folder}/friendsOfFriends.json"))
allFriends.update(read_json(f"{data_folder}/followersOfFollowers.json"))
allFriends.update(read_json(f"{data_folder}/followersLucaThomas.json"))
allFriends.update(read_json(f"{data_folder}/friendsLucaThomas.json"))
#serialize_json(data_folder,"allFriends.json", allFriends)
#pprint.PrettyPrinter().pprint(allFriends)

vector = []
for (k,v) in allFriends.items():
    for item in v:
        if item not in vector:
            vector.append(item)

serialize_json(data_folder,"NODUPLICATI.json", vector)

puntoQuattro = {}

for user in vector:
    json_data = api.get_user(user)._json
    foundInfo = {}
    foundInfo ["id"] = json_data["id"]
    foundInfo ["name"] = json_data["name"]
    foundInfo ["screen_name"] = json_data["screen_name"]
    foundInfo ["location"] = json_data["location"]
    foundInfo ["followers_count"] = json_data["followers_count"]
    foundInfo ["friends_count"] = json_data["friends_count"]
    puntoQuattro[user] = foundInfo
serialize_json(data_folder,"puntoQuattro.json", puntoQuattro)


Data read fron path: data/friendsOfFriends.json
Data read fron path: data/followersOfFollowers.json
Data read fron path: data/followersLucaThomas.json
Data read fron path: data/friendsLucaThomas.json
Data serialized to path: data/NODUPLICATI.json
Data serialized to path: data/puntoQuattro.json


In [18]:
def get_friendship(sourceid, targetid, api):
    friendship = api.show_friendship(source_screen_name=sourceid, target_screen_name=targetid)
    dictio = {"source_id":sourceid,"targe_id":targetid,"relation":"no"}
    if friendship[0].followed_by:
        dictio["relation"] = "targetToSource"
    if friendship[0].following:
        dictio["relation"] = "SourceToTarget"
    return dictio

files = read_json(f"{data_folder}/NODUPLICATI.json")

prova = []
for source in files:
    for target in files:
        prova.append(get_friendship(source,target,api))

pp = pprint.PrettyPrinter()
pp.pprint(prova)


serialize_json(data_folder,"prova.json", prova)

graph = nx.DiGraph()
graph.add_nodes_from("files")



"""
allDataPoint4 = read_json(f"{data_folder}/puntoQuattro.json")
graph = nx.DiGraph()
graph.add_nodes_from(allDataPoint4)
graph.graph["members"] = ["Luca","Thomas","Agustin","Edoardo"]

for user in allDataPoint4:
    for user2 in allDataPoint4:
        friendship = api.show_friendship(source_screen_name = user,target_screen_name = user2)
        if friendship[0].followed_by:
            graph.add_edge(user2,user)
        if friendship[0].following:
            graph.add_edge(user,user2)

nx.write_gpickle(graph,"data/graph.pkl")
"""

Data read fron path: data/NODUPLICATI.json
[{'relation': 'no',
  'source_id': 'crispy_mcbacon',
  'targe_id': 'crispy_mcbacon'},
 {'relation': 'SourceToTarget',
  'source_id': 'crispy_mcbacon',
  'targe_id': 'ciccio_feltri'},
 {'relation': 'SourceToTarget',
  'source_id': 'crispy_mcbacon',
  'targe_id': 'lucabazzetto'},
 {'relation': 'no', 'source_id': 'crispy_mcbacon', 'targe_id': 'elonmusk'},
 {'relation': 'SourceToTarget',
  'source_id': 'ciccio_feltri',
  'targe_id': 'crispy_mcbacon'},
 {'relation': 'no', 'source_id': 'ciccio_feltri', 'targe_id': 'ciccio_feltri'},
 {'relation': 'SourceToTarget',
  'source_id': 'ciccio_feltri',
  'targe_id': 'lucabazzetto'},
 {'relation': 'no', 'source_id': 'ciccio_feltri', 'targe_id': 'elonmusk'},
 {'relation': 'SourceToTarget',
  'source_id': 'lucabazzetto',
  'targe_id': 'crispy_mcbacon'},
 {'relation': 'SourceToTarget',
  'source_id': 'lucabazzetto',
  'targe_id': 'ciccio_feltri'},
 {'relation': 'no', 'source_id': 'lucabazzetto', 'targe_id': 'lu

'\nallDataPoint4 = read_json(f"{data_folder}/puntoQuattro.json")\ngraph = nx.DiGraph()\ngraph.add_nodes_from(allDataPoint4)\ngraph.graph["members"] = ["Luca","Thomas","Agustin","Edoardo"]\n\nfor user in allDataPoint4:\n    for user2 in allDataPoint4:\n        friendship = api.show_friendship(source_screen_name = user,target_screen_name = user2)\n        if friendship[0].followed_by:\n            graph.add_edge(user2,user)\n        if friendship[0].following:\n            graph.add_edge(user,user2)\n\nnx.write_gpickle(graph,"data/graph.pkl")\n'